# Residual Networks (ResNet)

:label:`sec_resnet`


As we design increasingly deeper networks it becomes imperative to understand how adding layers can increase the complexity and expressiveness of the network. Even more important is the ability to design networks where adding layers makes networks strictly more expressive rather than just different. To make some progress we need a bit of theory.

## Function Classes

Consider $\mathcal{F}$, the class of functions that a specific network architecture (together with learning rates and other hyperparameter settings) can reach. That is, for all $f \in \mathcal{F}$ there exists some set of parameters $W$ that can be obtained through training on a suitable dataset. Let us assume that $f^*$ is the function that we really would like to find. If it is in $\mathcal{F}$, we are in good shape but typically we will not be quite so lucky. Instead, we will try to find some $f^*_\mathcal{F}$ which is our best bet within $\mathcal{F}$. For instance, we might try finding it by solving the following optimization problem:

$$f^*_\mathcal{F} := \mathop{\mathrm{argmin}}_f L(X, Y, f) \text{ subject to } f \in \mathcal{F}.$$

It is only reasonable to assume that if we design a different and more powerful architecture $\mathcal{F}'$ we should arrive at a better outcome. In other words, we would expect that $f^*_{\mathcal{F}'}$ is "better" than $f^*_{\mathcal{F}}$. However, if $\mathcal{F} \not\subseteq \mathcal{F}'$ there is no guarantee that this should even happen. In fact, $f^*_{\mathcal{F}'}$ might well be worse. This is a situation that we often encounter in practice---adding layers does not only make the network more expressive, it also changes it in sometimes not quite so predictable ways. :numref:`fig_functionclasses`illustrates this in slightly abstract terms.

![Left: non-nested function classes. The distance may in fact increase as the complexity increases. Right: with nested function classes this does not happen.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/functionclasses.svg)

:label:`fig_functionclasses`



Only if larger function classes contain the smaller ones are we guaranteed that increasing them strictly increases the expressive power of the network. This is the question that He et al, 2016 considered when working on very deep computer vision models. At the heart of ResNet is the idea that every additional layer should contain the identity function as one of its elements. This means that if we can train the newly-added layer into an identity mapping $f(\mathbf{x}) = \mathbf{x}$, the new model will be as effective as the original model. As the new model may get a better solution to fit the training dataset, the added layer might make it easier to reduce training errors. Even better, the identity function rather than the null $f(\mathbf{x}) = 0$ should be the simplest function within a layer.

These considerations are rather profound but they led to a surprisingly simple
solution, a residual block. With it, :cite:`He.Zhang.Ren.ea.2016` won the ImageNet Visual
Recognition Challenge in 2015. The design had a profound influence on how to
build deep neural networks.


## Residual Blocks

Let us focus on a local neural network, as depicted below. Denote the input by $\mathbf{x}$. We assume that the ideal mapping we want to obtain by learning is $f(\mathbf{x})$, to be used as the input to the activation function. The portion within the dotted-line box in the left image must directly fit the mapping $f(\mathbf{x})$. This can be tricky if we do not need that particular layer and we would much rather retain the input $\mathbf{x}$. The portion within the dotted-line box in the right image now only needs to parametrize the *deviation* from the identity, since we return $\mathbf{x} + f(\mathbf{x})$. In practice, the residual mapping is often easier to optimize. We only need to set $f(\mathbf{x}) = 0$. The right image in :numref:`fig_residual_block` illustrates the basic Residual Block of ResNet. Similar architectures were later proposed for sequence models which we will study later.

![The difference between a regular block (left) and a residual block (right). In the latter case, we can short-circuit the convolutions.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/residual-block.svg)

:label:`fig_residual_block`



ResNet follows VGG's full $3\times 3$ convolutional layer design. The residual block has two $3\times 3$ convolutional layers with the same number of output channels. Each convolutional layer is followed by a batch normalization layer and a ReLU activation function. Then, we skip these two convolution operations and add the input directly before the final ReLU activation function. This kind of design requires that the output of the two convolutional layers be of the same shape as the input, so that they can be added together. If we want to change the number of channels or the stride, we need to introduce an additional $1\times 1$ convolutional layer to transform the input into the desired shape for the addition operation. Let us have a look at the code below.

In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.util.Training

In [2]:
import ai.djl.basicdataset.cv.classification.*;
// import org.apache.commons.lang3.ArrayUtils;

In [3]:
class Residual(numChannels: Int, use1x1Conv: Boolean, strideShape: Shape) :
    AbstractBlock(VERSION) {
    var block: ParallelBlock

    init {
        val b1: Block
        val conv1x1: Block
        b1 = SequentialBlock()
        b1.add(
            Conv2d.builder()
                .setFilters(numChannels)
                .setKernelShape(Shape(3, 3))
                .optPadding(Shape(1, 1))
                .optStride(strideShape)
                .build()
        )
            .add(BatchNorm.builder().build())
            .add(Activation::relu)
            .add(
                Conv2d.builder()
                    .setFilters(numChannels)
                    .setKernelShape(Shape(3, 3))
                    .optPadding(Shape(1, 1))
                    .build()
            )
            .add(BatchNorm.builder().build())
        if (use1x1Conv) {
            conv1x1 = SequentialBlock()
            conv1x1.add(
                Conv2d.builder()
                    .setFilters(numChannels)
                    .setKernelShape(Shape(1, 1))
                    .optStride(strideShape)
                    .build()
            )
        } else {
            conv1x1 = SequentialBlock()
            conv1x1.add(Blocks.identityBlock())
        }
        block = addChildBlock(
            "residualBlock",
            ParallelBlock(
                { list: List<NDList> ->
                    val unit = list[0]
                    val parallel = list[1]
                    NDList(
                        unit.singletonOrThrow()
                            .add(parallel.singletonOrThrow())
                            .ndArrayInternal
                            .relu()
                    )
                },
                mutableListOf(b1 as Block, conv1x1)
            )
        )
    }

    override fun toString(): String {
        return "Residual()"
    }

    override fun forwardInternal(
        parameterStore: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        return block.forward(parameterStore, inputs, training)
    }

    override fun getOutputShapes(inputs: Array<Shape>): Array<Shape> {
        var current: Array<Shape> = inputs
        for (block in block.children.values()) {
            current = block.getOutputShapes(current)
        }
        return current
    }

    override fun initializeChildBlocks(manager: NDManager, dataType: DataType, vararg inputShapes: Shape) {
        block.initialize(manager, dataType, *inputShapes)
    }

    companion object {
        private const val VERSION: Byte = 2
    }
}

This code generates two types of networks: one where we add the input to the output before applying the ReLU nonlinearity whenever `use1x1Conv` is `true`, and one where we adjust channels and resolution by means of a $1 \times 1$ convolution before adding. :numref:`fig_resnet_block` illustrates this:

![Left: regular ResNet block; Right: ResNet block with 1x1 convolution](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/resnet-block.svg)

:label:`fig_resnet_block`


Now let us look at a situation where the input and output are of the same shape.

In [4]:
    val manager = NDManager.newBaseManager()

    var blk = SequentialBlock()
    blk.add(Residual(3, false, Shape(1, 1)))

    var X = manager.randomUniform(0f, 1.0f, Shape(4, 3, 6, 6))

    val parameterStore = ParameterStore(manager, true)

    blk.initialize(manager, DataType.FLOAT32, X.shape)

    println(blk.forward(parameterStore, NDList(X), false).singletonOrThrow().shape)



(4, 3, 6, 6)


We also have the option to halve the output height and width while increasing the number of output channels.

In [5]:
    blk = SequentialBlock()
    blk.add(Residual(6, true, Shape(2, 2)))

    blk.initialize(manager, DataType.FLOAT32, X.shape)

    println(blk.forward(parameterStore, NDList(X), false).singletonOrThrow().shape)

(4, 6, 3, 3)


## ResNet Model

The first two layers of ResNet are the same as those of the GoogLeNet we described before: the $7\times 7$ convolutional layer with 64 output channels and a stride of 2 is followed by the $3\times 3$ maximum pooling layer with a stride of 2. The difference is the batch normalization layer added after each convolutional layer in ResNet.

In [6]:
    val net = SequentialBlock()
    net
        .add(
            Conv2d.builder()
                .setKernelShape(Shape(7, 7))
                .optStride(Shape(2, 2))
                .optPadding(Shape(3, 3))
                .setFilters(64)
                .build()
        )
        .add(BatchNorm.builder().build())
        .add(Activation::relu)
        .add(Pool.maxPool2dBlock(Shape(3, 3), Shape(2, 2), Shape(1, 1)))

SequentialBlock {
	Conv2d
	BatchNorm
	LambdaBlock
	maxPool2d
}

GoogLeNet uses four blocks made up of Inception blocks. However, ResNet uses four modules made up of residual blocks, each of which uses several residual blocks with the same number of output channels. The number of channels in the first module is the same as the number of input channels. Since a maximum pooling layer with a stride of 2 has already been used, it is not necessary to reduce the height and width. In the first residual block for each of the subsequent modules, the number of channels is doubled compared with that of the previous module, and the height and width are halved.

Now, we implement this module. Note that special processing has been performed on the first module.

In [7]:
fun resnetBlock(numChannels: Int, numResiduals: Int, firstBlock: Boolean): SequentialBlock {
    val blk = SequentialBlock()
    for (i in 0 until numResiduals) {
        if (i == 0 && !firstBlock) {
            blk.add(Residual(numChannels, true, Shape(2, 2)))
        } else {
            blk.add(Residual(numChannels, false, Shape(1, 1)))
        }
    }
    return blk
}


Then, we add all the residual blocks to ResNet. Here, two residual blocks are used for each module.

In [8]:
    net
        .add(resnetBlock(64, 2, true))
        .add(resnetBlock(128, 2, false))
        .add(resnetBlock(256, 2, false))
        .add(resnetBlock(512, 2, false))

SequentialBlock {
	Conv2d
	BatchNorm
	LambdaBlock
	maxPool2d
	SequentialBlock {
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock {
					identity
				}
			}
		}
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock {
					identity
				}
			}
		}
	}
	SequentialBlock {
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock {
					Conv2d
				}
			}
		}
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock {
					identity
				}
			}
		}
	}
	SequentialBlock {
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock

Finally, just like GoogLeNet, we add a global average pooling layer, followed by the fully connected layer output.

In [9]:
    net
        .add(Pool.globalAvgPool2dBlock())
        .add(Linear.builder().setUnits(10).build())

SequentialBlock {
	Conv2d
	BatchNorm
	LambdaBlock
	maxPool2d
	SequentialBlock {
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock {
					identity
				}
			}
		}
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock {
					identity
				}
			}
		}
	}
	SequentialBlock {
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock {
					Conv2d
				}
			}
		}
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock {
					identity
				}
			}
		}
	}
	SequentialBlock {
		Residual {
			residualBlock {
				SequentialBlock {
					Conv2d
					BatchNorm
					LambdaBlock
					Conv2d
					BatchNorm
				}
				SequentialBlock

There are 4 convolutional layers in each module (excluding the $1\times 1$ convolutional layer). Together with the first convolutional layer and the final fully connected layer, there are 18 layers in total. Therefore, this model is commonly known as ResNet-18. By configuring different numbers of channels and residual blocks in the module, we can create different ResNet models, such as the deeper 152-layer ResNet-152. Although the main architecture of ResNet is similar to that of GoogLeNet, ResNet's structure is simpler and easier to modify. All these factors have resulted in the rapid and widespread use of ResNet. :numref:`fig_ResNetFull` is a diagram of the full ResNet-18.

![ResNet 18](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/resnet18.svg)

:label:`fig_ResNetFull`


Before training ResNet, let us observe how the input shape changes between different modules in ResNet. As in all previous architectures, the resolution decreases while the number of channels increases up until the point where a global average pooling layer aggregates all features.

In [10]:
    X = manager.randomUniform(0f, 1f, Shape(1, 1, 224, 224))
    net.initialize(manager, DataType.FLOAT32, X.shape)
    var currentShape = X.shape

    for (i in 0 until net.children.size()) {
        X = net.children[i].value.forward(parameterStore, NDList(X), false).singletonOrThrow()
        currentShape = X.shape
        println(net.children[i].key + " layer output : " + currentShape)
    }

01Conv2d layer output : (1, 64, 112, 112)
02BatchNorm layer output : (1, 64, 112, 112)
03LambdaBlock layer output : (1, 64, 112, 112)
04LambdaBlock layer output : (1, 64, 56, 56)
05SequentialBlock layer output : (1, 64, 56, 56)
06SequentialBlock layer output : (1, 128, 28, 28)
07SequentialBlock layer output : (1, 256, 14, 14)
08SequentialBlock layer output : (1, 512, 7, 7)
09LambdaBlock layer output : (1, 512)
10Linear layer output : (1, 10)


## Data Acquisition and Training

We train ResNet on the Fashion-MNIST dataset, just like before. The only thing that has changed is the learning rate that decreased again, due to the more complex architecture.

In [11]:
    val batchSize = 256
    val lr = 0.05f
    val numEpochs = Integer.getInteger("MAX_EPOCH", 10)

    val epochCount = IntArray(numEpochs) { it + 1 }

    val trainIter = FashionMnist.builder()
        .addTransform(Resize(96))
        .addTransform(ToTensor())
        .optUsage(Dataset.Usage.TRAIN)
        .setSampling(batchSize, true)
        .optLimit(java.lang.Long.getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build()

    val testIter = FashionMnist.builder()
        .addTransform(Resize(96))
        .addTransform(ToTensor())
        .optUsage(Dataset.Usage.TEST)
        .setSampling(batchSize, true)
        .optLimit(java.lang.Long.getLong("DATASET_LIMIT", Long.MAX_VALUE))
        .build()

    trainIter.prepare()
    testIter.prepare()

    val model: Model = Model.newInstance("cnn")
    model.block = net

    val loss: Loss = Loss.softmaxCrossEntropyLoss()

    val lrt: Tracker = Tracker.fixed(lr)
    val sgd: Optimizer = Optimizer.sgd().setLearningRateTracker(lrt).build()

    val config = DefaultTrainingConfig(loss).optOptimizer(sgd) // Optimizer (loss function)
        .addEvaluator(Accuracy()) // Model Accuracy
        .addTrainingListeners(*TrainingListener.Defaults.logging()) // Logging

    val trainer: Trainer = model.newTrainer(config)


In [12]:
    val evaluatorMetrics: MutableMap<String, DoubleArray> = mutableMapOf()
    val avgTrainTimePerEpoch = Training.trainingChapter6(trainIter, testIter, numEpochs, trainer, evaluatorMetrics)

Training:    100% |████████████████████████████████████████| Accuracy: 0.82, SoftmaxCrossEntropyLoss: 0.61
Validating:  100% |████████████████████████████████████████|
Training:     99% |████████████████████████████████████████| Accuracy: 0.90, SoftmaxCrossEntropyLoss: 0.27

The execution was interrupted

In [13]:
    val trainLoss = evaluatorMetrics.get("train_epoch_SoftmaxCrossEntropyLoss")
    val trainAccuracy = evaluatorMetrics.get("train_epoch_Accuracy")
    val testAccuracy = evaluatorMetrics.get("validate_epoch_Accuracy")

    print("loss %.3f,".format(trainLoss!![numEpochs - 1]))
    print(" train acc %.3f,".format(trainAccuracy!![numEpochs - 1]))
    print(" test acc %.3f\n".format(testAccuracy!![numEpochs - 1]))
    println("%.1f examples/sec".format(trainIter.size() / (avgTrainTimePerEpoch / Math.pow(10.0, 9.0))))

null
java.lang.NullPointerException
	at Line_74.<init>(Line_74.jupyter-kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

![Contour Gradient Descent.](https://d2l-java-resources.s3.amazonaws.com/img/chapter_convolution-modern-cnn-resnet.png)

In [14]:
val trainLossLabel =  Array<String>(trainLoss!!.size) { "train loss" }
val trainAccLabel = Array<String>(trainLoss!!.size) { "train acc" }
val testAccLabel = Array<String>(trainLoss!!.size) { "test acc" }
val data = mapOf<String, Any>(
      "label" to trainLossLabel + trainAccLabel + testAccLabel,
      "epoch" to epochCount + epochCount + epochCount,
      "metrics" to trainLoss!! + trainAccuracy!! + testAccuracy!!
)

var plot = letsPlot(data)
plot += geomLine { x = "epoch" ; y = "metrics" ; color = "label"}
plot + ggsize(700, 500)

null
java.lang.NullPointerException
	at Line_75.<init>(Line_75.jupyter-kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:105)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:47)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnost

## Summary

* Residual blocks allow for a parametrization relative to the identity function $f(\mathbf{x}) = \mathbf{x}$.
* Adding residual blocks increases the function complexity in a well-defined manner.
* We can train an effective deep neural network by having residual blocks pass through cross-layer data channels.
* ResNet had a major influence on the design of subsequent deep neural networks, both for convolutional and sequential nature.


## Exercises

1. Refer to Table 1 in the :cite:`He.Zhang.Ren.ea.2016` to
   implement different variants.
1. For deeper networks, ResNet introduces a "bottleneck" architecture to reduce
   model complexity. Try to implement it.
1. In subsequent versions of ResNet, the author changed the "convolution, batch
   normalization, and activation" architecture to the "batch normalization,
   activation, and convolution" architecture. Make this improvement
   yourself. See Figure 1 in :cite:`He.Zhang.Ren.ea.2016*1`
   for details.
1. Prove that if $\mathbf{x}$ is generated by a ReLU, the ResNet block does indeed include the identity function.
1. Why cannot we just increase the complexity of functions without bound, even if the function classes are nested?